In [ ]:
# display some images for every different expression

import numpy as np
import seaborn as sns
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os

# size of the image: 48*48 pixels
pic_size = 48

# input path for the images
base_path = "H:/sample_images/images/"

plt.figure(0, figsize=(12,20))
cpt = 0

for expression in os.listdir(base_path + "train/"):
    for i in range(1,6):
        cpt = cpt + 1
        plt.subplot(7,5,cpt)
        img = load_img(base_path + "train/" + expression + "/" +os.listdir(base_path + "train/" + expression)[i], target_size=(pic_size, pic_size))
        plt.imshow(img, cmap="gray")

plt.tight_layout()
plt.show()

In [ ]:
for expression in os.listdir(base_path + "train"):
    print(str(len(os.listdir(base_path + "train/" + expression))) + " " + expression + " images")

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# number of images to feed into the NN for every batch
batch_size = 128

datagen_train = ImageDataGenerator()
datagen_validation = ImageDataGenerator()

train_generator = datagen_train.flow_from_directory(base_path + "train",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = datagen_validation.flow_from_directory(base_path + "validation",
                                                    target_size=(pic_size,pic_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

In [ ]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# number of possible label values
nb_classes = 7

# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time

from keras.callbacks import ModelCheckpoint

epochs = 20

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(
    x=train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=callbacks_list
)

# Save the model in the recommended Keras format
model.save("my_model.keras")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

# Load your saved model
from keras.models import load_model
loaded_model = load_model("my_model.keras")

# Generate predictions on the validation set
validation_generator.reset()
predictions = loaded_model.predict(validation_generator, steps=len(validation_generator), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
print("Accuracy:", accuracy_score(true_classes, predicted_classes))
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Generate and print confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing 
from sklearn.metrics import accuracy_score

images = []
labels = []

# Load the dataset
def load_images_from_folder(folder): 
    for expression in os.listdir(folder):
        for filename in os.listdir(os.path.join(folder, expression)):
            img = load_img(os.path.join(folder, expression, filename), target_size=(pic_size, pic_size), color_mode="grayscale")
            img_array = np.array(img)
            images.append(img_array.flatten())  # Flatten the image
            labels.append(expression)
    return np.array(images), np.array(labels)

train_images, train_labels = load_images_from_folder(os.path.join(base_path, "train"))
validation_images, validation_labels = load_images_from_folder(os.path.join(base_path, "validation"))

# Flatten and normalize the image data
scaler = preprocessing.MinMaxScaler()
train_images_scaled = scaler.fit_transform(train_images)
validation_images_scaled = scaler.transform(validation_images)

# Initialize and train the decision tree classifier
tree_classifier = DecisionTreeClassifier(random_state=42)
tree_classifier.fit(train_images_scaled, train_labels)

# Predict on validation set
predicted_labels = tree_classifier.predict(validation_images_scaled)

# Generate classification report and confusion matrix
report = classification_report(validation_labels, predicted_labels, target_names=os.listdir(os.path.join(base_path, "train")))
conf_matrix = confusion_matrix(validation_labels, predicted_labels)

print("Accuracy:", accuracy_score(validation_labels, predicted_labels))
print(report)
print(conf_matrix)


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

# Load images and labels
data = []
labels = []

# Iterate through each expression folder
for expression in os.listdir(base_path + "train/"):
    for img_path in os.listdir(base_path + "train/" + expression):
        img = Image.open(os.path.join(base_path, "train", expression, img_path)).convert('L')  # Open image in grayscale
        img = img.resize((pic_size, pic_size))  # Resize image to desired size
        img_array = np.array(img)  # Convert image to numpy array
        data.append(img_array)
        labels.append(expression)

# Convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Normalize pixel values to range [0, 1]
data = data / 255.0

# Encode labels to one-hot vectors
label_binarizer = LabelBinarizer()
labels_one_hot = label_binarizer.fit_transform(labels)

# Splitting the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels_one_hot, test_size=0.2, random_state=42)

# Create a Deep Belief Network model
model = Sequential()

model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1:])))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32))

# Output layer for multi-class classification
num_classes = len(np.unique(labels))  # Assuming labels contain class information
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 20
batch_size = 128

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_val, y_val), verbose=1)


In [ ]:
# Predict on validation data
y_pred = model.predict(X_val)

# Reshape y_pred to match the shape of y_val
num_classes = len(np.unique(labels))  # Assuming labels contain class information
y_pred_reshaped = np.reshape(y_pred, (-1, num_classes))

# Convert predicted and true labels back from one-hot encoding
y_true_labels = np.argmax(y_val, axis=1)
y_pred_labels = np.argmax(y_pred_reshaped, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_true_labels, y_pred_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true_labels, y_pred_labels)

# Calculate classification report
# report = classification_report(y_true_labels, y_pred_labels, target_names=label_binarizer.classes_)
report = classification_report(y_true_labels, y_pred_labels, target_names=label_binarizer.classes_, zero_division=1)

print("Accuracy:", accuracy)
print(report)
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
import os
import numpy as np
from keras.preprocessing import image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

images = []
labels = []

# Function to load and preprocess images
def load_images_and_labels(base_path):
    for expression in os.listdir(base_path + "train/"):
        for img_name in os.listdir(base_path + "train/" + expression):
            img = image.load_img(base_path + "train/" + expression + "/" + img_name, target_size=(pic_size, pic_size), color_mode="grayscale")
            img_array = image.img_to_array(img)
            images.append(img_array.flatten())
            labels.append(expression)
    return np.array(images), np.array(labels)

# Load images and labels
X, y = load_images_and_labels(base_path)

# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the Random Forest model
rf_model.fit(X_train, y_train)

# Predict on the validation set
y_pred = rf_model.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

# Generate classification report
print(classification_report(y_val, y_pred))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_val, y_pred)
print(conf_matrix)

In [ ]:
# show the confusion matrix of our predictions

# compute predictions
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)
y_test = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()